In [17]:
import torch
import sys
sys.path.insert(1, '../')

In [18]:
from models.resnet_simclr import ResNetSimCLR
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np

In [19]:
batch_size = 256
out_dim = 128

In [20]:
def _load_stl10(prefix="train"):
    X_train = np.fromfile('../data/stl10_binary/' + prefix + '_X.bin', dtype=np.uint8)
    y_train = np.fromfile('../data/stl10_binary/' + prefix + '_y.bin', dtype=np.uint8)

    X_train = np.reshape(X_train, (-1, 3, 96, 96))
    X_train = np.transpose(X_train, (0, 3, 2, 1))
    print("{} images".format(prefix))
    print(X_train.shape)
    print(y_train.shape)
    return X_train, y_train

In [21]:
# load STL-10 train data
X_train, y_train = _load_stl10("train")

train images
(5000, 96, 96, 3)
(5000,)


In [22]:
# load STL-10 test data
X_test, y_test = _load_stl10("test")

test images
(8000, 96, 96, 3)
(8000,)


## Test protocol #1 PCA features

In [23]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [24]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train.reshape((X_train.shape[0],-1)))

pca = PCA(n_components=128)

X_train_pca = pca.fit_transform(scaler.transform(X_train.reshape(X_train.shape[0], -1)))
X_test_pca = pca.transform(scaler.transform(X_test.reshape(X_test.shape[0], -1)))

print("PCA features")
print(X_train_pca.shape)
print(X_test_pca.shape)

PCA features
(5000, 128)
(8000, 128)


In [25]:
clf = LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', C=1.0)
clf.fit(X_train_pca, y_train)
print("PCA feature evaluation")
print("Train score:", clf.score(X_train_pca, y_train))
print("Test score:", clf.score(X_test_pca, y_test))

PCA feature evaluation
Train score: 0.4306
Test score: 0.3625


In [26]:
data_augment = transforms.Compose([transforms.RandomResizedCrop(96),
                                   transforms.ToTensor()])

train_dataset = datasets.STL10('../data', split='train', download=True, transform=data_augment)
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=1, drop_last=False, shuffle=False)

Files already downloaded and verified


In [27]:
test_dataset = datasets.STL10('../data', split='test', download=True, transform=data_augment)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, drop_last=False, shuffle=False)

Files already downloaded and verified


In [28]:
model = ResNetSimCLR(out_dim=out_dim)
model.eval()
print(model)

state_dict = torch.load('../checkpoints/checkpoint.pth')
print(state_dict.keys())

model.load_state_dict(state_dict)

ResNetSimCLR(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

<All keys matched successfully>

## Protocol #2 Linear separability evaluation

In [29]:
X_train_feature = []

for step, (batch_x, batch_y) in enumerate(train_loader):
    features, _ = model(batch_x)
    X_train_feature.extend(features.detach().numpy())
    
X_train_feature = np.array(X_train_feature)

print("Train features")
print(X_train_feature.shape)

Train features
(5000, 512)


In [30]:
X_test_feature = []

for step, (batch_x, batch_y) in enumerate(test_loader):
    features, _ = model(batch_x)
    X_test_feature.extend(features.detach().numpy())
    
X_test_feature = np.array(X_test_feature)

print("Test features")
print(X_test_feature.shape)

Test features
(8000, 512)


In [31]:
clf = LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', C=1.0)

scaler = preprocessing.StandardScaler()
scaler.fit(X_train_feature)

clf.fit(scaler.transform(X_train_feature), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
print("SimCLR feature evaluation")
print("Train score:", clf.score(scaler.transform(X_train_feature), y_train))
print("Test score:", clf.score(scaler.transform(X_test_feature), y_test))

SimCLR feature evaluation
Train score: 0.8914
Test score: 0.6425
